#Yolo v5

In [32]:
!pip install torchvision==0.13.0
!pip install torch==1.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import torch
from PIL import Image, ImageGrab
from torchvision.transforms import ToTensor

In [34]:
print(torch.version.__version__)

1.12.1+cu113


In [35]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

img_path = '/content/we.jpg'
imgs = Image.open(img_path) 

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-7 Python-3.7.14 torch-1.12.1+cu113 CPU



  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


### Настройки вывода

In [36]:
# Inference Settings
yolo_model.conf = 0.3  # порог уверенности модели для фильтрации предсказаний 
yolo_model.iou = 0.4  # пороговое значение NMS IoU
yolo_model.agnostic = False  # обнаружение объектов, не зависящих от класса
yolo_model.multi_label = False  # несколько классов на один bbox
yolo_model.classes = [0]  # выбор классов из COCO датасета, нулевой класс = persons
yolo_model.max_det = 10  # максимальное количество обнаружений на одно изображение
yolo_model.amp = True  # атоматический вывод смешанной точности

In [37]:
%%time

yolo_results = yolo_model(imgs)
yolo_results.print()
yolo_results.save() 

image 1/1: 1280x962 1 person
Speed: 46.0ms pre-process, 358.0ms inference, 2.0ms NMS per image at shape (1, 3, 640, 512)
Saved 1 image to runs/detect/exp4


CPU times: user 443 ms, sys: 33.1 ms, total: 476 ms
Wall time: 475 ms


In [38]:
coordinates = yolo_results.xyxy[0][:, :4]
yolo_results.pandas().xyxy[0] 

,xmin,ymin,xmax,ymax,confidence,class,name
0,32.18457,9.024048,939.7052,1235.446655,0.368445,0,person


## Resume
➕Плюсы: самая быстрая, точная и легко запускающаяся модель; широкий выбор параметров настройки вывода

➖Минусы: низкая уверенность модели в прогнозах 

## Сравнение версий
### ✅yolov5s 
Уверенность: 0,37

Скорость: 7.2ms pre-process, 341.7ms inference, 1.4ms NMS per image, **total: 476 ms** 
### ✅yolov5m
Уверенность: 0,68

Скорость: 31.0ms pre-process, 897.9ms inference, 1.7ms NMS per image, **total: 1 s**
### ✅yolov5l
Уверенность: 0,69

Скорость: 32.1ms pre-process, 1793.0ms inference, 1.3ms NMS per image, **total:1.88 s**

🏅Обнаружила второго человека по половинке лица с вероятность 0,43

#FasterRCNN

In [49]:
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_320_fpn, FasterRCNN_MobileNet_V3_Large_320_FPN_Weights
import urllib
from torchvision import transforms
from torchvision.io.image import read_image
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from torchvision.transforms.functional import pil_to_tensor

In [50]:
weights = FasterRCNN_MobileNet_V3_Large_320_FPN_Weights.DEFAULT
resnet_model = fasterrcnn_mobilenet_v3_large_320_fpn(weights=weights, pretrained=True, progress=False)
resnet_model.eval()
preprocess = weights.transforms()

Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth


In [51]:
img_path = '/content/we.jpg'
img = read_image(img_path)

batch = [preprocess(img)]

In [52]:
weights.meta["categories"] = weights.meta["categories"][0:2]
weights.meta["categories"]

['__background__', 'person']

In [53]:
prediction

{'boxes': tensor([[  10.95274,   24.38524,  806.90668, 1238.53479]], grad_fn=<IndexBackward0>),
 'labels': tensor([1]),
 'scores': tensor([0.99129], grad_fn=<IndexBackward0>)}

In [54]:
%%time
prediction = resnet_model(batch)[0]
prediction["boxes"] = prediction["boxes"][prediction["scores"] > 0.96]
prediction["labels"] = prediction["labels"][prediction["scores"] > 0.96]
prediction["scores"] = prediction["scores"][prediction["scores"] > 0.96]

labels = [weights.meta["categories"][i] for i in prediction["labels"]]
box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                          labels=labels,
                          colors="cyan",
                          width=2, 
                          font_size = 30)

# to_pil_image(box)
res = Image.fromarray(box.permute(1, 2, 0).contiguous().numpy())
res.save("bbox_faster.png")

/usr/local/lib/python3.7/dist-packages/torchvision/utils.py:237: UserWarning: Argument 'font_size' will be ignored since 'font' is not set.
  warnings.warn("Argument 'font_size' will be ignored since 'font' is not set.")


CPU times: user 738 ms, sys: 13.5 ms, total: 751 ms
Wall time: 758 ms


## Resume:
➕Плюсы: высокая точность и уверенность модели в прогнозах (>0,96); FasterRCNN_ResNet50_FPN_V2 обнаруживает с высокой вероятностью человека лишь по части лица в отличие от yolo

➖Минусы: долгая обработка; отсутствие настраиваемых параметров помимо классов

## Сравнение версий
### ✅FasterRCNN_ResNet50_V2
🏅Распознала второе лицо

Уверенность: 0.97+

Скорость: **total 15.6 s**
### ✅FasterRCNN_MobileNet_V3_Large
Уверенность: 0.99, но не распознал второе лицо

Скорость: **total: 1.81 s**
### ✅FasterRCNN_MobileNet_V3_Large_320
Уверенность: 0.99, но распознал два лица за одно (bbox шире)

Скорость: **total: 751 ms**

# RetinaNet

In [55]:
from torchvision.models.detection import retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt

In [56]:
weights = RetinaNet_ResNet50_FPN_V2_Weights.DEFAULT
model = retinanet_resnet50_fpn_v2(weights=weights, score_thresh=0.35)
model.eval()
preprocess = weights.transforms()

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth


  0%|          | 0.00/146M [00:00<?, ?B/s]

In [57]:
img_path = '/content/we.jpg'
img = read_image(img_path)

batch = [preprocess(img)]

In [58]:
weights.meta["categories"] = weights.meta["categories"][0:2]
weights.meta["categories"]

['__background__', 'person']

In [60]:
prediction

{'boxes': tensor([[   0.00000,    0.00000,  947.76874, 1255.99988]], grad_fn=<IndexBackward0>),
 'scores': tensor([0.94286], grad_fn=<IndexBackward0>),
 'labels': tensor([1])}

In [59]:
%%time
prediction = model(batch)[0]
prediction["boxes"] = prediction["boxes"][prediction["scores"] > 0.9]
prediction["labels"] = prediction["labels"][prediction["scores"] > 0.9]
prediction["scores"] = prediction["scores"][prediction["scores"] > 0.9]

labels = [weights.meta["categories"][i] for i in prediction["labels"]]
box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                          labels=labels,
                          colors="cyan",
                          width=2, 
                          font_size = 30)

# to_pil_image(box)
res = Image.fromarray(box.permute(1, 2, 0).contiguous().numpy())
res.save("bbox_retina.png")

/usr/local/lib/python3.7/dist-packages/torchvision/utils.py:237: UserWarning: Argument 'font_size' will be ignored since 'font' is not set.
  warnings.warn("Argument 'font_size' will be ignored since 'font' is not set.")


CPU times: user 8.37 s, sys: 1 s, total: 9.38 s
Wall time: 9.42 s


## Resume:
➕Плюсы: высокая уверенность модели в проигнозируемом объекте, но ниже, чем у FasterRCNN

➖Минусы: не обнаружен второй объект; затраченное время не соответствует качеству

## Сравнение версий
### ✅RetinaNet_ResNet50_FPN_V2
Уверенность: 0.94

Скорость: **total: 9.38 s**